In [1]:
import pandas as pd
import numpy as np
import folium

import io
from PIL import Image
from selenium import webdriver
import imageio
from folium.features import DivIcon
from folium.plugins import HeatMap

# Data Loading and Processing

In [37]:
df = pd.read_csv('臺北捷運每日分時各站OD流量統計資料_201905.csv')


In [38]:
df = df.rename(columns={"日期": "date", '時段':'time', '進站':'in', '出站':'out', '人次':'count'})

In [39]:
df_in = df.groupby(by=['time', 'in']).agg('sum').reset_index()
df_out = df.groupby(by=['time', 'out']).agg('sum').reset_index()

/var/folders/59/p50xxbtd4y5f88p88xxyc8bw0000gn/T/ipykernel_2368/2100274649.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_in = df.groupby(by=['time', 'in']).agg('sum').reset_index()
/var/folders/59/p50xxbtd4y5f88p88xxyc8bw0000gn/T/ipykernel_2368/2100274649.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_out = df.groupby(by=['time', 'out']).agg('sum').reset_index()


In [5]:
# df_out.to_csv('taipei_metro_out.csv', encoding='utf_8_sig')
# df_in.to_csv('taipei_metro_in.csv', encoding='utf_8_sig')

In [40]:
df_inout = pd.merge(df_in, df_out, left_on=['time', 'in'], right_on=['time', 'out'])

In [41]:
df_inout = df_inout.rename(columns={"count_x": "count_in", "count_y": "count_out"})
df_inout['net'] = df_inout['count_out'] - df_inout['count_in']
df_inout

,time,in,count_in,out,count_out,net
0,0,BL板橋,6725,BL板橋,8308,1583
1,0,七張,1499,七張,5616,4117
2,0,三和國中,1172,三和國中,5952,4780
3,0,三民高中,987,三民高中,5782,4795
4,0,三重,1626,三重,1921,295
...,...,...,...,...,...,...
2263,23,頂埔,1493,頂埔,10986,9493
2264,23,頂溪,12629,頂溪,35238,22609
2265,23,頭前庄,1637,頭前庄,3314,1677
2266,23,麟光,1222,麟光,3889,2667


In [44]:
df_station = pd.read_csv('Geo-info_Stations.csv')
# df_station.to_csv('northern-taiwan.csv', encoding='utf_8_sig')

In [51]:
df_station.replace('板橋', 'BL板橋', inplace=True)

In [53]:
df_merge = pd.merge(df_inout, df_station, left_on='in', right_on='station_name_tw', how='left')

In [54]:
df_merge.dropna(inplace=True)

In [55]:
df_merge = df_merge.iloc[:, np.r_[:3, 4,5, 14:16]]

In [56]:
df_merge.describe()

,time,count_in,count_out,net,lat,lon
count,2646.000000,2646.000000,2646.000000,2646.000000,2646.000000,2646.000000
mean,12.714286,35280.333711,35340.534769,60.201058,25.046133,121.522291
std,6.548813,48865.562784,49845.319038,29342.108027,0.041428,0.044695
min,0.000000,0.000000,0.000000,-185262.000000,24.957855,121.411270
25%,8.000000,7183.000000,7700.750000,-6881.250000,25.023777,121.498633
50%,13.000000,20907.000000,20915.000000,0.000000,25.042356,121.523041
75%,18.000000,44033.000000,45206.750000,7163.500000,25.063000,121.551098
max,23.000000,455872.000000,513015.000000,264336.000000,25.167818,121.617500


In [13]:
# biggest = df_merge.select_dtypes(include=[np.number]).max().max()

# Plotting - Heatmap



In [57]:

def make_heatmap(df, time):
    df_zoom = df[df.time == time]

    latitude = 25.041629
    longitude = 121.543767

    # Create map and display it
    taipei_map = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='cartodbpositron')
    
    data_net = list(map(list, zip(df_zoom['lat'], df_zoom['lon'], df_zoom['net'])))

    # gradient={.3: 'gray', .5: 'lime', 1: 'red'}

    HeatMap(data_net).add_to(taipei_map)

    folium.map.Marker(
        [25.107940, 121.582397],
        icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 20pt">{time}:00</div>',
        )).add_to(taipei_map)

    taipei_map

    img_data = taipei_map._to_png(3)
    img = Image.open(io.BytesIO(img_data))
    img.save(f'{time}.png')

In [58]:
for i in df_merge.time.unique():
    make_heatmap(df_merge, i)



# Plotting - Circle color map

In [91]:
def make_ccmap(df, time):
    df_zoom = df[df.time == time]
    
    df_merge['color'] = ['red' if x >= 0 else 'green' for x in df_merge['net']]

    latitude = 25.041629
    longitude = 121.543767

    # Create map and display it
    taipei_map = folium.Map(location=[latitude, longitude], zoom_start=12, tiles='cartodbpositron')
    
    for name,row in df_zoom.iterrows():
        radius = row['net']/100
        folium.Circle(location=(row['lat'], row['lon']), 
                    radius= radius, 
                     color=row['color'], 
                     fill=True, 
                     fill_opacity=0.1)\
        .add_to(taipei_map) 

    folium.map.Marker(
        [25.107940, 121.582397],
        icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 20pt">{time}:00</div>',
        )).add_to(taipei_map)
    folium.map.Marker(
        [25.117161, 121.581152],
        icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 20pt">Taipei Metro</div>',
        )).add_to(taipei_map)
    folium.map.Marker(
        [25.106303, 121.396567],
        icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12pt; color:red">Red: Net Inflow</div>',
        )).add_to(taipei_map)
    folium.map.Marker(
        [25.086409, 121.396567],
        icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12pt; color:green">Green: Net Outflow</div>',
        )).add_to(taipei_map)
    folium.map.Marker(
        [24.971658, 121.582397],
        icon=DivIcon(
        icon_size=(250,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12pt">Credit: Chung-Hao Lee</div>',
        )).add_to(taipei_map)

    taipei_map

    img_data = taipei_map._to_png(3)
    img = Image.open(io.BytesIO(img_data))
    img.save(f'{time}.png')

In [92]:
for i in df_merge.time.unique():
    make_ccmap(df_merge, i)

# Visualization GIF

In [93]:

with imageio.get_writer(uri='taipei_metro.gif', mode='I', fps=1.5) as writer:
    for i in df_merge.time.unique():
        writer.append_data(imageio.imread(f'{i}.png'))

/var/folders/59/p50xxbtd4y5f88p88xxyc8bw0000gn/T/ipykernel_2368/1758974825.py:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  writer.append_data(imageio.imread(f'{i}.png'))
